In [11]:
"""
SrTiO3 space group: P m -3 m (221)
Sr -> 1a
Ti -> 1b
O -> 3c
Sr = xu.materials.element.Sr
Ti = xu.materials.element.Ti
O = xu.materials.element.O
SrTiO3 = xu.materials.Crystal("SrTiO3", xu.materials.SGLattice(221, 3.905, atoms=[Sr, Ti, O], pos=['1a', '1b', '3c']),)

LaAlO3 space group: P m -3 m (167)
La -> 6a
Al -> 6b
O -> 18e
La = xu.materials.element.La
Al = xu.materials.element.Al
O = xu.materials.element.O
LaAlO3 - xu.materials.Crystal("LaAlO3", xu.materials.SGLattice(221, 3.792, atoms=[La, Al, O], pos=['1a', '1b', '3c']))
"""
#print(rsmFile.scanaxis)

TwoThetaOmega


In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 26 11:00:40 2018

In this version I graph the output as H K L parameters

@author: oliver
"""
%matplotlib widget

import os #this deals with file directories
import xrayutilities as xu #this is the main package that is used
import numpy as np #useful mathematics package
import tkinter as tk #user interface for file IO
import xrdtools #easy way for reading XRDML
from tkinter import filedialog #file dialogue from TKinter
import matplotlib.pyplot as plt #plotting package
import matplotlib as mpl #plotting
#%matplotlib notebook
#%matplotlib inline

#import mpld3
#mpld3.enable_notebook()


# plot settings for matplotlib
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 20.0
mpl.rcParams['axes.labelsize'] = 'large'
mpl.rcParams['figure.subplot.bottom'] = 0.16
mpl.rcParams['figure.subplot.left'] = 0.17
mpl.rcParams['savefig.dpi'] = 200
mpl.rcParams['axes.grid'] = False


nchannel = 255
chpdeg = nchannel/2.511 #2.511 degrees is angular acceptance of xrays to detector
center_ch = 128
energy = 1240/0.154




def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

class rsm(object):
    
    def __init__(self):
        # omega 2theta and data arrays. Omega and 2theta are 1D arrays while data is a 2D array with dimensions of omega and 2theta
        self.omega = []
        self.tt = []
        self.data = []
        self.meta = []
        #test files for dev
        self.xrdmlTestFile = 'REAL1203_RSM_220_perp.xrdml'
        self.rasTestFile = 'N0044dOP_RSM_310.ras'
        #theoretical omega and 2theta positions for a given substrate peak
        self.theoOm = []
        self.theoTT = []
        self.qx = []
        self.qy = []
        self.qz = []
        self.omCorr = []
        self.ttCorr = []
        self.gridder = []
        self.rsmRef = []
        self.iHKL = []
        self.oHKL = []
        self.h = 0
        self.k = 0
        self.l = 0
        self.maxInt = 0#maximum intensity of RSM peak
        self.filepath = self.openDialogue() # open GUI for user to select file
        #self.filepath = self.xrdmlTestFile
        #self.filepath = \
        #r'C:\Users\olive\OneDrive - UNSW\Experiments 2019\Shintaro HTXRD RSM\20190124_Dan_AL5119c\AL5119c-103_02_0002_0350-1_K.ras'
        b = self.filepath.index('.') # find the period that separates filename and extension
        ext = self.filepath[(b+1):] # get extension and determine whether it is RAS or XRDML
        
        a = self.filepath.rfind('/')  # find the period that separates the directory and filename
        self.filename = self.filepath[(a+1):(b)]
        
        if ext == 'xrdml':
            (self.omega, self.tt, self.data) = self.xrdml_file(self, self.filepath)
            print('omega = ' + str(self.omega.shape))
            print('2theta = ' + str(self.tt.shape))
            print('data = ' + str(self.data.shape))
        elif ext == 'ras':
            (self.omega, self.tt, self.data) = self.ras_file(self, self.filepath)
        else:
            print('filetype not supported.')
        
        
        print(self.data.shape)
        [self.h, self.k, self.l] = input('Substrate reflection?').split(' ')
        print(str(self.h) + ' ' + str(self.k) + ' ' + str(self.l))
        self.rsmRef = (int(self.h), int(self.k), int(self.l))
        print('rsmRef = ' + str(self.rsmRef))
            
        [self.substrate, self.hxrd] = self.initSubstrate()
        
        [self.qx, self.qy, self.qz] = self.alignSubstratePeak()
        #[self.omCorr, self.ttCorr] = self.hxrd.Q2Ang([self.qx, self.qy, self.qz])
        
        
        
    @staticmethod
    def xrdml_file(self, file):
        #file = self.openDialogue()
        #print(file)
        data = xrdtools.read_xrdml(file)
        #data['Omega'] = data['Omega'][:,0]
        #data['2Theta'] = data['2Theta'][0]
        #om, tt = np.meshgrid(data['Omega'], data['2Theta'])
        om = data['Omega']
        tt = data['2Theta']
        
        return (om,
                tt,
                data['data'])
    @staticmethod
    def ras_file(self, file):
        # Read RAS data to object
        rasFile = xu.io.rigaku_ras.RASFile(file)
        
        self.scanaxis = rasFile.scans[1].scan_axis
        self.stepSize = rasFile.scans[1].meas_step
        self.measureSpeed= rasFile.scans[1].meas_speed
        self.dataCount = rasFile.scans[1].length
        # Read raw motor position and intensity data to large 1D arrays
        
        '''
            Can i get more data out from data['variable']? Can I adjust things to deal with 
            2theta-omega scans better?
        '''
        #print(rasFile.scans[1].scan_axis)
        omttRaw, data = xu.io.getras_scan(rasFile.filename+'%s', '', self.scanaxis)
        #print('omttRaw shape is ' + str(omttRaw.shape))
        inte = data['int']
        #print(data['TwoThetaOmega'])
        npinte = np.array(inte)
        intensities = npinte.reshape(len(rasFile.scans), rasFile.scans[0].length)
        # Read omega data from motor positions at the start of each 2theta-Omega scan
        om = [rasFile.scans[i].init_mopo['Omega'] for i in range(0, len(rasFile.scans))]
        # Convert 2theta-omega data to 1D array
        tt = [omttRaw.data[i] for i in range(0, rasFile.scans[0].length)]
        
        if self.scanaxis == 'TwoThetaOmega': # If RSM is 2theta/omega vs omega scan, adjust omega values in 2D matrix
            omga = [[om[i] + (n * self.stepSize/2) for n in range(0,len(tt))] for i in range(0,len(om))]
            omga = np.array(omga)
            #print('length of omega = ' + str(len(om)))
            ttheta = np.array(tt)
            #print(len(ttheta))
            tt = [[ttheta[i] for i in range(0,len(ttheta))] for j in range(0, len(omga))]
            tt = np.array(tt)
            #print('ras_file adjusted tt shape = ' + str(tt.shape))
            #print('ras_file adjusted omega shape = ' + str(omga.shape))
            #print('ras_file adjusted int shape = ' + str(intensities.shape))

        #self.motor = rasFile.scans[1].moname
        
        return (np.transpose(omga), np.transpose(tt), np.transpose(intensities))
    
    def openDialogue(self):
        root = tk.Tk()
        root.withdraw()
        filepath = filedialog.askopenfilename()
        filename = os.path.basename(filepath)
        return filepath
    
    def plot2d(self, ax=None, **kwargs):
        if ax is None:
            fig, ax = plt.subplots()
        #plt.figure(figsize=[12,10])
        
        #listData= self.data.tolist()
        
        #ax.imshow(self.omega,self.tt, np.transpose(np.log10(self.data)).tolist(), cmap='jet', origin='lower', **kwargs)
        om = np.array(self.omega)
        tt = np.array(self.tt)
        if(om.ndim == 1 and tt.ndim == 1):
            om, tt = np.meshgrid(self.omega, self.tt)
        elif (om.ndim != tt.ndim):
            print('Error. omega and twotheta arrays must have same dimension. dim(omega) = ' \
                  + str(om.shape) + 'and dim(tt) = ' + str(tt.shape))
        '''    
        if self.scanaxis == "TwoThetaOmega":
            om1D = np.ravel(om)
            tt1D = np.ravel(tt)
            dat1D = np.ravel(self.data)
            ax.scatter(om1D,tt1D,c=dat1D,cmap='jet')
        '''    
        #
        #a = ax.contour(om,tt, np.log10(self.data).tolist(), cmap='jet', origin='lower', **kwargs)
        #a = ax.contourf(om,tt, np.log10(self.data), cmap='jet')
        a = ax.contourf(np.transpose(om[0,:]),np.transpose(tt[:,0]), np.log10(self.data), cmap='jet')
        #ax.clabel(a, inline=True, fontsize=10)
        #ax.colorbar(label='Data')
        ax.set_xlabel(r'$\omega$')
        ax.set_ylabel(r'2$\theta$-$\omega$')
        return ax
    
    def plotQ(self, xGrid, yGrid, ax=None, **kwargs):
        #if ax is None:
        #    fig, ax = plt.subplots()
        plt.figure(figsize=[7,5])
        plt.title( self.filename )
        
        
        self.gridder = xu.Gridder2D(xGrid,yGrid)
        self.gridder(self.qy, self.qz, self.data)
        intHigh = np.argmax(self.data)
        dynhigh = np.rint(np.log10(intHigh))
        #INT = xu.maplog(self.gridder.data.transpose(), 3, -1)
        plt.contourf(self.gridder.xaxis, self.gridder.yaxis, np.transpose(np.log10(self.gridder.data)), cmap='jet', **kwargs)
        #ax.colorbar(label='Data')
        plt.xlabel(r'$Q_{[001]}$')
        plt.ylabel(r'$Q_{[110]}$')
        
        return ax

    def to_csv(self, fname=None):
        if fname is None:
            fname = 'some_placeholder.csv'
        np.savetxt(fname, list(zip(self.omege, self.tt, self.tt)), delimiter=',')
        
    ''' Peakfitting-related functions (maybe make another class??)'''
    
    def findSubstratePeak(self, axis1, axis2, data):
        ''' Returns indices of axis1 and axis2 where the substrate peak is positioned (based off maximum counts)'''
        maxIndex = np.argmax(data)
        return np.unravel_index( maxIndex, (len(data[:,0]), len(data[0,:])))

    def initSubstrate(self):
        La = xu.materials.elements.La
        Al = xu.materials.elements.Al
        O = xu.materials.elements.O2m
        Sr = xu.materials.elements.Sr
        Ti = xu.materials.elements.Ti
        Ta = xu.materials.elements.Ta
        
        substrateMat = []
        while (substrateMat != 'LAO' and substrateMat != 'STO' and substrateMat != 'LSAT'):
            substrateMat = input('Sample substrate (LAO, STO or LSAT)?')
            print(substrateMat)
            print('input valid substrate material')
            
            
        [ipH, ipK, ipL] = input("Input in-plane direction of substrate without spaces (i.e. 100, 110, etc): ").split(' ')
        ipHKL = [ipH, ipK, ipL]
        [opH, opK, opL] = input("Input out-of-plane direction of substrate without spaces (i.e. 001, 110, 210 etc): ").split(' ')
        opHKL = [opH, opK, opL]
        """
        substrateMat = "LAO"
        ipHKL = (1, 0, 0)
        opHKL = (0, 0, 1)
        """
        self.iHKL = ipHKL
        self.oHKL = opHKL
        if substrateMat == "LAO":
            substrate = xu.materials.Crystal("LaAlO3", xu.materials.SGLattice(221, 3.784, \
                            atoms=[La, Al, O], pos=['1a', '1b', '3c']))
            hxrd = xu.HXRD(substrate.Q(int(ipHKL[0]), int(ipHKL[1]), int(ipHKL[2])), \
                           substrate.Q(int(opHKL[0]), int(opHKL[1]), int(opHKL[2])), en=energy, geometry = 'lo_hi')
        elif substrateMat == "STO":
            substrate = xu.materials.SrTiO3
            #substrate = xu.materials.Crystal("SrTiO3", xu.materials.SGLattice(221, 3.905, atoms=[Sr, Ti, O], \
            #pos=['1a', '1b', '3c']))
            hxrd = xu.HXRD(substrate.Q(int(ipHKL[0]), int(ipHKL[1]), int(ipHKL[2])), \
                           substrate.Q(int(opHKL[0]), int(opHKL[1]), int(opHKL[2])), en=energy)
        elif substrateMat == "LSAT": # need to make an alloy of LaAlO3 and Sr2AlTaO6
            mat1 = xu.materials.Crystal("LaAlO3", xu.materials.SGLattice(221, 3.79, \
                           atoms=[La, Al, O], pos=['1a', '1b', '3c']))
            mat2 = xu.materials.Crystal("Sr2AlTaO6", xu.materials.SGLattice(221, 3.898,\
                           atoms=[Sr, Al, Ta, O], pos=['8c', '4a', '4b', '24c']))
            substrate = xu.materials.CubicAlloy(mat1, mat2, 0.71)
            hxrd = xu.HXRD(substrate.Q(int(ipHKL[0]), int(ipHKL[1]), int(ipHKL[2])), \
                           substrate.Q(int(opHKL[0]), int(opHKL[1]), int(opHKL[2])), en=energy)
           
        return [substrate, hxrd]
    
    def alignSubstratePeak(self):
        
        indexMax = np.argmax(self.data)
        self.maxInt = indexMax
        print('max index is ' + str(indexMax))
        data = np.array(self.data)
        #print('data shape = ' + str(data.shape))
        tupleIndex = np.unravel_index( indexMax, (len(data[:,0]), len(data[0,:])) )
        #print('tupleIndex = ' + str(tupleIndex))
        
        
        [self.theoOm, dummy, dummy, self.theoTT] = self.hxrd.Q2Ang(self.substrate.Q(self.rsmRef))
        expTT = self.tt[tupleIndex[0],tupleIndex[1]]
        expOm = self.omega[tupleIndex[0],tupleIndex[1]]
        print('experimental omega = ' + str(expOm))
        print('experimental tt = ' + str(expTT))
        print('theoretical omega = ' + str(self.theoOm))
        print('theoretical 2theta = ' + str(self.theoTT))
        print('omega shape = ' + str(self.omega.shape))
        #print('expt omega = ' + str(self.omega[tupleIndex[1], tupleIndex[0]]))
        omega = []
        tt = []
        
        offset = (self.theoTT/2) + self.theoOm
        omnominal = (self.theoTT/2) + offset
        self.hxrd.Ang2Q.init_linear('y+', center_ch, nchannel, chpdeg=chpdeg) 
        '''
        #converts omega and 2theta for wrapped 1D arrays to plot in Origin
        
        for i in range(0,len(self.omega)):
            for j in range(0,len(self.tt)):
                omega.append(self.omega[i])
                tt.append(self.tt[j])
        '''
        #[omega, tt] = np.meshgrid(self.omega, self.tt)
        #print('shape of omega is' + str(np.array(omega).shape))
        #print((self.omega[tupleIndex[0]]+ tupleIndex[1] * self.stepSize/2) )
        delta = (self.omega[tupleIndex[0], tupleIndex[1]] - self.theoOm, self.tt[tupleIndex[0], tupleIndex[1]] - self.theoTT)
        print('delta = ' + str(delta))
        
        [qx, qy, qz] = self.hxrd.Ang2Q(self.omega, self.tt, delta=[expOm - self.theoOm, expTT - self.theoTT] )
        
        #print('delta = ' + str(delta))
        #[qx, qy, qz] = self.hxrd.Ang2Q(self.omega, self.tt, delta = [ -0.031223667727843463, -0.11234498129975634 ])
        #[qx, qy, qz] = self.hxrd.Ang2Q(self.omega, self.tt, delta = [ 0,0 ])
        return [qx, qy, qz]

    

In [ ]:
rsmFile = rsm()
print('done')
#fig = rsmFile.plot2d()



rsmFile.plotQ()

In [23]:
rsmFile.plotQ( 100, 101, levels=20)

FigureCanvasNbAgg()

In [13]:
plt.contourf(rsmFile.tt, rsmFile.omega, np.log10(rsmFile.data), cmap='jet')

In [77]:
bMax = np.argmax(rsmFile.gridder.data[xMin:xMax, yMin:yMax])
print(bMax)
tupleIndex = np.unravel_index( substrateMax, (len(rsmFile.gridder.xaxis), len(rsmFile.gridder.yaxis)) )
print(rsmFile.gridder.data[tupleIndex])

219
15550.5


In [9]:
ax, h = xu.materials.show_reciprocal_space_plane(rsmFile.substrate, rsmFile.hxrd, ttmax=140)

KeyError: '8c'

In [243]:
#QxSub = float(input('Qx of substrate?'))
#QzSub = float(input('Qz of substrate?'))
QxFilm = float(input('Qx of film?'))
QzFilm = float(input('Qz of film?'))

c1 = 0
c2 = 0

#coeffs = np.matrix([[c1], [c2]])
b1 = np.matrix(rsmFile.iHKL, dtype='float').T
b2 = np.matrix(rsmFile.oHKL, dtype='float').T
b3 = np.cross(np.transpose(b1), np.transpose(b2))
b3 = np.transpose(b3)
basisV = np.c_[b1, b2, b3]
vec = np.matrix(rsmFile.rsmRef)
coeffs = np.linalg.inv(basisV) * np.transpose(vec)

if coeffs[0] == 0:
    coeffs[0] = 1

#ipSub = 2*np.pi/(QxSub/int(coeffs[0]))
#opSub = 2*np.pi/(QzSub/int(coeffs[1]))
if Qx != 0:
    ipFilm = 2*np.pi/(QxFilm/int(coeffs[0])) * np.sqrt(rsmFile.iHKL[0] + rsmFile.iHKL[1] + rsmFile.iHKL[2])
    print('Film IP ' + str(rsmFile.iHKL) + ' = ' + str(ipFilm))
    
opFilm = 2*np.pi/(QzFilm/int(coeffs[1])) * np.sqrt(rsmFile.oHKL[0] + rsmFile.oHKL[1] + rsmFile.oHKL[2]) * np.sqrt(2)

print('substrate ' + str(rsmFile.iHKL) + ' = ' + str(ipSub))
print('substrate ' + str(rsmFile.oHKL) + ' = ' + str(opSub))


print('Film OOP ' + str(rsmFile.oHKL) + ' = ' + str(opFilm))

#plt.figure(figsize=[7,5])
#plt.plot(T, a)
#plt.show()
#plt.xlabel('Temperature (K)')
#plt.ylabel('a lattice parameter (Angstrom)')

Qx of film? 3.24514
Qz of film? 4.44696


substrate ['0', '0', '1'] = 3.8680156656352587
substrate ['1', '1', '0'] = 2.7343102988067765
Film IP ['0', '0', '1'] = 3.872366250565206
Film OOP ['1', '1', '0'] = 5.651667932411883


indexMax = np.argmax(rsmFile.gridder.data)
print(rsmFile.gridder.data.shape)
tupleIndex = np.unravel_index( indexMax, (len(rsmFile.gridder.data[:,0]), len(rsmFile.gridder.data[0,:])) )
print(tupleIndex)
print(rsmFile.gridder.xaxis[tupleIndex[0]])
print(rsmFile.gridder.yaxis[tupleIndex[1]])
print(rsmFile.gridder.data[tupleIndex])
print(2 * pi / rsmFile.gridder.yaxis[tupleIndex[1]])
#print(len(rsmFile.omega))
#print(len(rsmFile.tt))

In [66]:
qRange = (1.6, 1.8, 3.8, 4,2)
#bfoQzRange = i in [rsmFile.gridder.]

substrateMax = np.argmax(rsmFile.gridder.data)
print('--------- Substrate Qx/Qz peak positions and corresponding lattice parameters ------------')
print(substrateMax)
tupleIndex = np.unravel_index( substrateMax, (len(rsmFile.gridder.xaxis), len(rsmFile.gridder.yaxis)) )
#print(tupleIndex)
print(rsmFile.gridder.xaxis[139])
print(rsmFile.gridder.yaxis[163])
qx = rsmFile.gridder.xaxis[tupleIndex[0]]
qz = rsmFile.gridder.yaxis[tupleIndex[1]]
da = 2* np.pi*rsmFile.rsmRef[0] /qx
dc = 2*np.pi*rsmFile.rsmRef[2]/qz
print('a = ' + str(da))
print('c = ' + str(dc))
print('--------- Film Qx/Qz peak positions and corresponding lattice parameters ----------------')

#xRange = [i in rsmFile.gridder.xaxis if (rsmFile.gridder.xaxis[i] > qRange[0] and rsmFile.gridder.xaxis[i] < qRange[1]) ]
#print(xRange)
xMin = find_nearest(rsmFile.gridder.xaxis, qRange[0])
xMin = rsmFile.gridder.xaxis.tolist().index(xMin)
print(rsmFile.gridder.xaxis[xMin])
xMax = find_nearest(rsmFile.gridder.xaxis, qRange[1])
xMax = rsmFile.gridder.xaxis.tolist().index(xMax)

yMin = find_nearest(rsmFile.gridder.yaxis, qRange[2])
yMin = rsmFile.gridder.yaxis.tolist().index(yMin)

yMax = find_nearest(rsmFile.gridder.yaxis, qRange[3])
yMax = rsmFile.gridder.yaxis.tolist().index(yMax)
print(yMax)
dataList = rsmFile.gridder.xaxis.tolist()
print(yMin)
#print(dataList.index(xMin))

bfoMax = np.argmax(rsmFile.gridder.data[xMin:xMax, yMin:yMax])
newMax = bfoMax + len(rsmFile.gridder.xaxis) * yMin + len(rsmFile.gridder.xaxis[0:xMin]) * (yMax - yMin) + \
len(rsmFile.gridder.xaxis) * 
print(bfoMax)

BFOtupleIndex = np.unravel_index(bfoMax, (len(rsmFile.gridder.xaxis), len(rsmFile.gridder.yaxis)))
BFOqx = rsmFile.gridder.xaxis[BFOtupleIndex[0]]
BFOqz = rsmFile.gridder.yaxis[BFOtupleIndex[1]]
print(BFOqx)
print(BFOqz)
BFOa = 2* np.pi*rsmFile.rsmRef[0] /BFOqx
BFOc = 2* np.pi*rsmFile.rsmRef[2] /BFOqz
print('BFO a = ' + str(BFOa))
print('BFO c = ' + str(BFOc))

--------- Substrate Qx/Qz peak positions and corresponding lattice parameters ------------
28102
1.659477139456134
4.976950828304485
a = 3.7862439667224312
c = 3.7873703341289193
--------- Film Qx/Qz peak positions and corresponding lattice parameters ----------------
1.597433659089058
54
32
219
0.708143773827635
3.6770122293103027
BFO a = 8.872753725162227
BFO c = 5.126323967944586


In [65]:
#LSAT = xu.materials.SGLattice(221, 3.868) # not including elements and Wyckoff positions makes the SGLattice not allow .Q
#sub = xu.materials.SrTiO3
La = xu.materials.elements.La3p
Al = xu.materials.elements.Al3p
O = xu.materials.elements.O2m
Sr = xu.materials.elements.Sr
Ti = xu.materials.elements.Ti
Ta = xu.materials.elements.Ta

sub = xu.materials.Crystal("LaAlO3", xu.materials.SGLattice(221, 3.784, atoms=[La, Al, O], pos=['1a', '1b', '3c']))
hxrd = xu.FourC([1, 1, 0], [0, 0, 1], en=energy)
ang = hxrd.Q2Ang(sub.Q(0,0,1))
print(ang)

[  11.73950651    0.         -180.           23.47901301]


In [45]:
print(rsmFile.measureSpeed)
print(60 / (rsmFile.measureSpeed / rsmFile.stepSize) )
omega = []
omega = [rsmFile.omega[i] + rsmFile.tt[j] for ]

20.0
0.06


In [6]:
indexMax = np.argmax(rsmFile.data)
print(rsmFile.data.shape)
tupleIndex = np.unravel_index( indexMax, (len(rsmFile.data[:,0]), len(rsmFile.data[0,:])) )
print(tupleIndex)
print(rsmFile.omega[tupleIndex[0]])
print(rsmFile.tt[tupleIndex[1]])
print(rsmFile.data[tupleIndex])
print(len(rsmFile.omega))
print(len(rsmFile.tt))


(241, 551)
(199, 502)
11.54
79.98
26982.0
241
551


In [ ]:
[sub, hxrd] = rsmFile.initSubstrate()
hsub = xu.HXRD(sub.Q(1,0,0), sub.Q(0,0,1))
ax, h = xu.materials.show_reciprocal_space_plane(sub, hsub, ttmax=160)
#ipHKL = np.array(0,0,1)
#opHKL = np.array(1,1,0)
#[substrate, hxrd] = rsmFile.initSubstrate(subMat, ipHKL, opHKL)



['hello']


In [13]:
xC = rsmFile.omega[tupleIndex[0]]
yC = rsmFile.tt[tupleIndex[1]]
xSigma = 0.001
ySigma = 0.001
amp = rsmFile.data[tupleIndex]
angle = 0
background = 1
p = [xC, yC, xSigma, ySigma, amp, background, angle]
drange=[62, 63, 77.5, 78.5]
om = np.array(rsmFile.omega)
tt = np.array(rsmFile.tt)
dat = np.array(rsmFile.data)

gaussFit = xu.math.functions.Gauss2d(rsmFile.omega[0], rsmFile.tt[0], *p)
fitParams, cov = xu.math.fit.fit_peak2d(om.flatten(), tt.flatten(), dat.flatten(), p, drange, xu.math.functions.Gauss2d)


ValueError: operands could not be broadcast together with shapes (167,) (314,) 

In [71]:
#p = [XCEN1, YCEN1, SIGMAX1, SIGMAY1, AMP1, BACKGROUND, ANGLE1]
gaussFit = xu.math.functions.Gauss2d(rsmFile.omega, rsmFile.tt, *p)
fitParams, cov = xu.math.fit.fit_peak2d(rsmFile.omega, rsmFile.tt, rsmFile.data, p, drange=[34, 35, 67, 69], gaussFit)

SyntaxError: positional argument follows keyword argument (<ipython-input-71-d5a4e5a582ea>, line 3)

In [ ]:
import matplotlib.pyplot as plt
'''
Code from stackoverflow to get the new x and y limits of an interactive matplotlib plot after the user has zoomed in
'''
#
# Some toy data
x_seq = [x / 100.0 for x in xrange(1, 100)]
y_seq = [x**2 for x in x_seq]

#
# Scatter plot
fig, ax = plt.subplots(1, 1)
ax.scatter(x_seq, y_seq)

#
# Declare and register callbacks
def on_xlims_change(axes):
    print "updated xlims: ", ax.get_xlim()

def on_ylims_change(axes):
    print "updated ylims: ", ax.get_ylim()

ax.callbacks.connect('xlim_changed', on_xlims_change)
ax.callbacks.connect('ylim_changed', on_ylims_change)

#
# Show
plt.show()

In [37]:
class rsm(object):
    
    def __init__(self, omega, tt, data, meta={}):
        self.omega = omega
        self.tt = tt
        self.data = data
        self.meta = meta
        #self.file = file
        self.xrdmlTestFile = 'RB18017_RSM_310_phi0.xrdml'
        self.rasTestFile = 'N0044dOP_RSM_310.ras'
    
    @staticmethod
    def read_file(filename):
        #filename = 'N0044dOP_RSM_310.ras'
        ext = filename.split('.')[-1]
        if ext == 'xrdml':
            #print(file)
            data = xrdtools.read_xrdml(filename)
            data['Omega'] = data['Omega'][:,0]
            data['2Theta'] = data['2Theta'][0]
            return rsm(data['Omega'],
                      data['2Theta'],
                      data['data'])
        
        elif ext == 'ras':
            # Read RAS data to object
            rasFile = xu.io.rigaku_ras.RASFile(file)
            omttRaw, data = xu.io.getras_scan(rasFile.filename+'%s', '', 'TwoThetaOmega')
            inte = data['int']
            npinte = np.array(inte)
            intensities = npinte.reshape(len(rasFile.scans), rasFile.scans[0].length)
            #print(intensities)
            # Read omega data from motor positions at the start of each 2theta-Omega scan
            om = [rasFile.scans[i].init_mopo['Omega'] for i in range(0, len(rasFile.scans))]
            #print(len(rasFile.scans))
            # Convert 2theta-omega data to 1D array
            tt = [omttRaw.data[i] for i in range(0, rasFile.scans[0].length)]
            return rsm(om, tt, intensities)
        else:
            raise ValueError('Filetype not supported')
            
    @staticmethod
    def read_dialogue():
        root = tk.Tk()
        root.withdraw()
        filepath = filedialog.askopenfilename()
        filename = os.path.basename(filepath)
        read_file(filename)
        
    def plot(self, ax=None, **kwargs):
        if ax is None:
            fig, ax = plt.subplots()
        plt.figure(figsize=[12,10])
        
        ax.contourf(self.omega,self.tt, np.transpose(np.log10(self.data)), cmap='jet', **kwargs)
        #ax.colorbar(label='Data')
        ax.set_xlabel(r'$\omega$')
        ax.set_ylabel(r'2$\theta$-$\omega$')
        return ax
    
    def to_csv(self, fname=None):
        if fname is None:
            fname = 'some_placeholder.csv'
        np.savetxt(fname, list(zip(self.omege, self.tt, self.tt)), delimiter=',')